In [1]:
import os
import sys
from importlib import reload
sys.path.append('../src')

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import tqdm

In [2]:
import transformers

In [3]:
import torch.utils.data as D

In [6]:
import dataset
import transforms

# Dataset

In [87]:
reload(dataset)
valid = dataset.TokenizerDataset('../input/validation.csv')

In [88]:
%%time
item = valid[35]

CPU times: user 3.09 ms, sys: 0 ns, total: 3.09 ms
Wall time: 2.72 ms


#### Pre-tokenized version

In [183]:
reload(dataset)
valid = dataset.Dataset('../input/validation.npz')

## Sampling

We will create weighted random sampler that will produce random samples with replacement for each epoch with approximately equal amount of positiva and negative samples

In [184]:
sampler = valid.weighted_sampler()

In [205]:
# rerun this cell to ensure that values will be different for each epoch
sampled = [i for i in sampler]
np.unique(valid.y[sampled], return_counts=True), len(sampled)

((array([0, 1], dtype=uint8), array([1228, 1232])), 2460)

## Text transforms

In [76]:
reload(transforms)

<module 'transforms' from '../src/transforms.py'>

In [93]:
sent = valid.df[valid.text_column][500]
sent

' Si racconta che in quel momento una voce dal Cielo proclamò: «Tu sei beato Akiva, il tuo respiro si è spento con  Echad . Tu sei beato Akiva, avrai una parte nel Mondo Avvenire.»[4]  Una frase simile non e  appropriata in un enciclopedia. Che una simile panzana sia stata raccontata e  fuori discussione, che si racconti ora, ridicolo.'

In [70]:
shuffle_sent = transforms.ShuffleSentences(p=1)
shuffle_sent(sent)

'Tu sei beato Akiva, avrai una parte nel Mondo Avvenire.»[4]  Una frase simile non e  appropriata in un enciclopedia.  Si racconta che in quel momento una voce dal Cielo proclamò: «Tu sei beato Akiva, il tuo respiro si è spento con  Echad . Che una simile panzana sia stata raccontata e  fuori discussione, che si racconti ora, ridicolo.'

In [71]:
%timeit _ = shuffle_sent(sent)

81.3 µs ± 663 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [73]:
shuffle_word = transforms.SwapWords(p=1, swap_p=0.1)
shuffle_word(sent)

'Si racconta che in quel momento una voce dal Cielo proclamò: «Tu beato sei Akiva, il tuo respiro si è spento con Echad . Tu sei beato Akiva, avrai una parte nel Avvenire.»[4] Mondo frase Una simile non e appropriata in un enciclopedia. Che una simile sia panzana stata raccontata fuori discussione, e che si ora, ridicolo. racconti'

In [74]:
%timeit _ = shuffle_word(sent)

152 µs ± 42.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [77]:
drop_word = transforms.DropWords(p=1, drop_p=0.1)
drop_word(sent)

'Si racconta che in quel momento voce dal Cielo proclamò: «Tu sei Akiva, il tuo respiro si è spento con Echad . Tu sei beato avrai una parte nel Mondo Avvenire.»[4] Una frase simile non appropriata in un enciclopedia. Che una simile panzana sia stata raccontata e fuori discussione, che si racconti ora, ridicolo.'

In [78]:
%timeit _ = drop_word(sent)

157 µs ± 45 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [80]:
all_transforms = transforms.Compose([
    transforms.ShuffleSentences(p=0.5),
    transforms.SwapWords(swap_p=0.1),
    transforms.DropWords(drop_p=0.1),
])
all_transforms(sent)

'Tu sei beato Akiva, avrai una parte nel Mondo Avvenire.»[4]  Una frase simile non e  appropriata in un enciclopedia.  Si racconta che in quel momento una voce dal Cielo proclamò: «Tu sei beato Akiva, il tuo respiro si è spento con  Echad . Che una simile panzana sia stata raccontata e  fuori discussione, che si racconti ora, ridicolo.'

In [81]:
%timeit _ = all_transforms(sent)

200 µs ± 49.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Token transforms 

In [128]:
reload(transforms)

<module 'transforms' from '../src/transforms.py'>

In [129]:
print(valid.df[valid.text_column][500])
sent = valid[500][0].numpy()
sent[:50]

 Si racconta che in quel momento una voce dal Cielo proclamò: «Tu sei beato Akiva, il tuo respiro si è spento con  Echad . Tu sei beato Akiva, avrai una parte nel Mondo Avvenire.»[4]  Una frase simile non e  appropriata in un enciclopedia. Che una simile panzana sia stata raccontata e  fuori discussione, che si racconti ora, ridicolo.


array([     0,    602,  54963,    290,     23,  19488,   3594,    220,
        26845,   1640,   2079,   8242,    502,  60397,   2516,     12,
           94,  15665,   7583,    186,   2832,  36851,    330,      4,
          211,   3860, 225565,     78,    565,  66933,     31,    158,
        74242,   8408,      6,      5,   1371,   7583,    186,   2832,
        36851,    330,      4,    185,   8287,    220,   1235,   1718,
       168054,   4908])

In [130]:
shuffle_sent = transforms.TokenShuffleSentences(p=1)
valid.tokenizer.decode(shuffle_sent(sent)[shuffle_sent.mask(sent)])

'Si racconta che in quel momento una voce dal Cielo proclamò: «Tu sei beato Akiva, il tuo respiro si è spento con Echad. Che una simile panzana sia stata raccontata e fuori discussione, che si racconti ora, ridicolo. Tu sei beato Akiva, avrai una parte nel Mondo Avvenire.»[] Una frase simile non e appropriata in un enciclopedia.'

In [123]:
%timeit _ = shuffle_sent(sent)

29.5 µs ± 4.63 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [131]:
shuffle_word = transforms.TokenSwapWords(p=1, swap_p=0.1)
valid.tokenizer.decode(shuffle_word(sent)[shuffle_word.mask(sent)])

'Si racconta che in quel momento una voce dalelo pro Ciclamò: «Tu sei beatova Aki, il respiro tuo si è spento con Echad. Tu sei beato Akiva, avrai una nel parte Mondo Avvenire.[»] Una frase simile non e appropriata in en unciclopedia. Che una simile panzana sia stata raccontata e fuorie discussion, che si racconti ora, ridicolo.'

In [132]:
%timeit _ = shuffle_word(sent)

1.4 ms ± 319 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [133]:
drop_word = transforms.TokenDropWords(p=1, drop_p=0.1)
valid.tokenizer.decode(drop_word(sent)[drop_word.mask(sent)])

'Si racconta in quel momento voce dal Ci proclamò: «Tu seiato Akiva, il tuo respiro si è spento Echad. Tu seiato Akiva, avrai una parte nel Mondo Avvenire.»[ Una frase simile non e appropriata in unciclopedia. Che una simile panzana sia stata raccontata e fuori discussione, che si racconti ora,olo.</s><pad><pad><pad><pad><pad><pad><pad><pad>'

In [134]:
%timeit _ = drop_word(sent)

1.38 ms ± 351 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [137]:
all_transforms = transforms.Compose([
    transforms.TokenShuffleSentences(p=0.5),
    transforms.TokenSwapWords(swap_p=0.1),
    transforms.TokenDropWords(drop_p=0.1),
])
valid.tokenizer.decode(all_transforms(sent)[drop_word.mask(sent)])

'Tu sei beato Akiva, avrai una parte nel Mondo Avvenire. Si racconta in quel momento una voce dal Cielo proclamò: «Tu beato Akiva, il tuo respiro si è spento con Echad. Che una simile panzana sia stata raccontata e fuorie, che si racconti ora, ridicolo.»[] Una frase simile non e appropriata in enciclopedia.</s><pad><pad><pad>'

In [138]:
%timeit _ = all_transforms(sent)

1.32 ms ± 380 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
